In [1]:
def solve_scal(x):
    sort_list = ['1-49人',
        '50-99人',
        '100-499人',
        '500-999人',
        '1000-2000人',
        '2000-5000人',
        '10000人以上',
        '5000-10000人'
        ]

    empty_list = []
    new_list = []
    for y in x:
        if y == sort_list[0]:
            empty_list.append(int(0))
        elif y == sort_list[1]:
            empty_list.append(int(1))
        elif y == sort_list[2]:
            empty_list.append(int(2))
        elif y == sort_list[3]:
            empty_list.append(int(3))
        elif y == sort_list[4]:
            empty_list.append(int(4))
        elif y == sort_list[5]:
            empty_list.append(int(5))
        elif y == sort_list[6]:
            empty_list.append(int(6))
        elif y == sort_list[7]:
            empty_list.append(int(7))
    get_list = sorted(empty_list)
    for i in range(len(get_list)):
        new_list.append(sort_list[get_list[i]])
    return  new_list

In [ ]:
def get_tree_graph(job_name,city_name):
    import pandas
    from requests_html import HTMLSession
    import json
    import pandas as pd
    import requests
    用户输入职位 = job_name
    用户输入城市 = city_name
    城市编码 = {
    '全国': '410',
        '北京': '010',
        '上海': '020',
        '天津': '030',
        '重庆': '040',
        '广州': '050020',
        '深圳': '050090',
        '苏州': '060080',
        '南京': '060020',
        '杭州': '070020',
        '大连': '210040',
        '成都': '280020',
        '武汉': '170020',
        '西安': '270020'

}
    url = "https://apic.liepin.com/api/com.liepin.searchfront4c.pc-search-job"
    payload = {
    "data": {
        "mainSearchPcConditionForm": {
            "city": 城市编码[用户输入城市],
            "dq": 城市编码[用户输入城市],
            "pubTime": "",
            "currentPage": 0,
            "pageSize": 40,
            "key": str(用户输入职位),
            "suggestTag": "",
            "workYearCode": "0",
            "compId": "",
            "compName": "",
            "compTag": "",
            "industry": "",
            "salary": "",
            "jobKind": "",
            "compScale": "",
            "compKind": "",
            "compStage": "",
            "eduLevel": ""
        },
        "passThroughForm": {
            "scene": "input",
            "skId": "",
            "fkId": "",
            "ckId": "h2c8pxojavrmo1w785z7ueih2ybfpux8",
            "suggest": None
        }
    }
}
    session = HTMLSession()
    headers = {your headers}
    r = session.post(url, data=json.dumps(payload), headers=headers)
    response_data = r.json()
    page = response_data['data']['pagination']['totalPage']
    response_df = []
    for i in range(page): # 需要判断页面的数据有多少页
        payload['data']['mainSearchPcConditionForm']['currentPage']=i
        # send a POST request with headers
        r = requests.post(url, data=json.dumps(payload), headers=headers)
    response_data = r.json()
    print(response_data)
    df = pd.json_normalize(response_data['data']['data']['jobCardList'])
    response_df.append(df)
    df = pd.concat(response_df)
    key = payload['data']['mainSearchPcConditionForm']['key']
    df.to_excel('liepin2.xlsx')
    df_all = pd.read_excel('liepin2.xlsx')
    df_all
    df_cities = df[['job.dq','comp.compScale','comp.compIndustry','comp.compName','job.title','job.salary']]
    df_cities
    df_cities = df_cities.rename(columns={
    'job.dq':'地区',
    'comp.compScale':'公司规模',
    'comp.compIndustry':'公司行业',
    'comp.compName':'公司名称',
    'job.title':'岗位名称',
    'job.salary':'工资',

    })
    set_dq = set(df_cities['地区'])
    list_dq = list(set_dq)
    list_dq
    a = len(list_dq)

    data_all = [{
        'name':'北京',
        'children':[
                    {'name':a,
                     'children':[
                         {
                            'name':"公司规模:"+b,
                             'children':[
                                 {
                                     'name': "公司行业:"+str(c),
                                     'children':[
                                         {
                                           'name': "公司名称:"+d,
                                           'children':[
                                               {
                                                'name': "岗位名称:"+e,
                                                'children':[
                                                    {
                                                       'name': "工资："+f,
                                                    } for f in list(set(df_cities[(df_cities['地区'] == a) & (df_cities['公司规模'] == b) & (df_cities['公司行业'] == c)  & (df_cities['公司名称'] == d) & (df_cities['岗位名称'] == e)]['工资']))
                                                ]
                                               } for e in list(set(df_cities[(df_cities['地区'] == a) & (df_cities['公司规模'] == b) & (df_cities['公司行业'] == c)  & (df_cities['公司名称'] == d)]['岗位名称']))
                                           ]
                                         } for d in list(set(df_cities[(df_cities['地区'] == a) & (df_cities['公司规模'] == b) & (df_cities['公司行业'] == c) ]['公司名称']))
                                     ]
                                  } for c in list(set(df_cities[(df_cities['地区'] == a) & (df_cities['公司规模'] == b)]['公司行业']))
                             ]
                         } for b in solve_scal(list(set(df_cities[(df_cities['地区'] == a)]['公司规模'])))
                     ]
                     }  for a in list(set(df_cities['地区']))
         ]
        }
    ]
    data_all
    from pyecharts import options as opts
    from pyecharts.charts import Tree
    c = (
        Tree()
        .add("", data_all, collapse_interval=1, layout="radial")
        .set_global_opts(title_opts=opts.TitleOpts(title="Tree-Layout"))
    )
    return c
